<a href="https://colab.research.google.com/github/kaviswetha-801/gpt2-text-generator-gradio/blob/main/GRADIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers gradio accelerate

In [3]:
import torch
import gradio as gr
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"


MODEL_NAME = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
model.to(device)
model.eval()

def generate_text(prompt, max_new_tokens=80, temperature=0.95):
    if not prompt.strip():
        return "Please enter a prompt."

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)


with gr.Blocks() as demo:
    gr.Markdown("## GPT-2 Text Generator (Google Colab Demo)")
    gr.Markdown("Fast, lightweight, and GPU-aware")

    prompt = gr.Textbox(
        label="Prompt",
        placeholder="Once upon a time in a world ruled by AI...",
        lines=3
    )

    with gr.Row():
        max_tokens = gr.Slider(20, 200, value=80, step=10, label="New Tokens")
        temperature = gr.Slider(0.3, 1.3, value=0.9, step=0.1, label="Temperature")

    generate = gr.Button("Generate")
    output = gr.Textbox(label="Generated Text", lines=10)

    generate.click(
        generate_text,
        inputs=[prompt, max_tokens, temperature],
        outputs=output
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d41401339d50049c8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
